## <p style="text-align: center;">Country Club PART 2: Python & SQLite</p>

### Import Library or Packages needed to be able to build and run these Python and SQL Commands

In [1]:
import sqlite3
import os
import pandas as pd

from sqlalchemy import create_engine


### Create a database connection to the SQLite database specified by the db_file

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn

### Query all rows in the tasks table

In [3]:
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
        SELECT *
        FROM FACILITIES
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


#### Create a database connection

In [4]:
def main():
    database = "sqlite_db_pythonsqlite.db"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks
(0, 'Tennis Court 1', 5, 25, 10000, 200)
(1, 'Tennis Court 2', 5, 25, 8000, 200)
(2, 'Badminton Court', 0, 15.5, 4000, 50)
(3, 'Table Tennis', 0, 5, 320, 10)
(4, 'Massage Room 1', 9.9, 80, 4000, 3000)
(5, 'Massage Room 2', 9.9, 80, 4000, 3000)
(6, 'Squash Court', 3.5, 17.5, 5000, 80)
(7, 'Snooker Table', 0, 5, 450, 15)
(8, 'Pool Table', 0, 5, 400, 15)


#### Create an engine 

In [5]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

### Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [6]:
query = """
SELECT
    sub2.name AS Facility_Name,
    sub2.totalrevenue AS Total_Revenue
FROM
    (SELECT sub1.facilityname AS NAME,
        SUM(sub1.revenue) AS totalrevenue
    FROM(SELECT B.bookid, F.name AS facilityname,
            CASE WHEN B.memid = 0 THEN(B.slots * F.guestcost) 
            ELSE B.slots * F.membercost
    END AS Revenue
FROM Bookings AS B
    LEFT JOIN Members AS M ON M.memid = B.memid
    LEFT JOIN Facilities AS F ON F.facid = B.facid
) AS sub1
GROUP BY sub1.facilityname
) AS sub2
GROUP BY Facility_Name
HAVING Total_Revenue < 1000
ORDER BY Total_Revenue DESC;
"""
Revenue_Under_1000 = pd.read_sql_query(query, engine)
Revenue_Under_1000

,Facility_Name,Total_Revenue
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order 

In [7]:
query = """
SELECT
    sub2.memberName AS Member,
    sub2.recommender_first || ', ' || sub2.recommender_last AS Recommended_By
FROM
    (
    SELECT
        sub1.memberName AS memberName,
        sub1.recommenderId AS memberId,
        M.firstname AS recommender_first,
        M.surname AS recommender_last
    FROM(SELECT
            M2.memid AS memberId,
            M1.firstname || ', ' || M1.surname AS memberName,
            M2.recommendedby AS recommenderId
        FROM Members AS M1
            INNER JOIN Members AS M2
                ON M1.memid = M2.memid
        WHERE(M2.recommendedby >= 1) AND M1.memid >= 1) AS sub1
    LEFT JOIN Members AS M
    ON sub1.recommenderId = M.memid
WHERE M.memid >= 1) AS sub2
ORDER BY sub2.recommender_last;
"""
Recommended_By = pd.read_sql_query(query, engine)
Recommended_By

,Member,Recommended_By
0,"Ramnaresh, Sarwin","Florence, Bader"
1,"Joan, Coplin","Timothy, Baker"
2,"Matthew, Genting","Gerald, Butters"
3,"Timothy, Baker","Jemima, Farrell"
4,"David, Pinker","Jemima, Farrell"
5,"Henrietta, Rumney","Matthew, Genting"
6,"Douglas, Jones","David, Jones"
7,"Nancy, Dare","Janice, Joplette"
8,"David, Jones","Janice, Joplette"
9,"John, Hunt","Millicent, Purview"


### Q12: Find the facilities with their usage by member, but not guests. 

In [8]:
query = """
SELECT 
    F.name AS Facilities, 
    SUM(b.slots) AS Usage 
FROM Bookings AS B 
    LEFT JOIN Facilities AS F 
        ON F.facid = B.facid 
    LEFT JOIN Members AS M 
        ON M.memid = B.memid 
WHERE B.memid != 0
GROUP BY Facilities 
ORDER BY Usage DESC;
"""
Member_Usage = pd.read_sql_query(query, engine)
Member_Usage

,Facilities,Usage
0,Badminton Court,1086
1,Tennis Court 1,957
2,Massage Room 1,884
3,Tennis Court 2,882
4,Snooker Table,860
5,Pool Table,856
6,Table Tennis,794
7,Squash Court,418
8,Massage Room 2,54


### Q13: Find the facilities usage by month, but not guests

In [9]:
query = """
SELECT sub.MONTH AS Month, sub.facilityname AS Facilities, SUM(sub.slotNumber) AS Usage 
FROM (SELECT 
          strftime('%m', starttime) AS MONTH, 
          F.name AS facilityname, 
          B.slots AS slotNumber 
      FROM Bookings AS B 
      LEFT JOIN Facilities AS F
          ON F.facid = B.facid
      LEFT JOIN Members AS M
          ON M.memid = B.memid 
      WHERE B.memid <> 0) sub 
GROUP BY Month, Facilities 
ORDER BY Month, Usage DESC;
"""
Facilities_Usage_ByMonth = pd.read_sql_query(query, engine)
Facilities_Usage_ByMonth

,Month,Facilities,Usage
0,07,Tennis Court 1,201
1,07,Massage Room 1,166
2,07,Badminton Court,165
3,07,Snooker Table,140
4,07,Tennis Court 2,123
5,07,Pool Table,110
6,07,Table Tennis,98
7,07,Squash Court,50
8,07,Massage Room 2,8
9,08,Badminton Court,414
